In [1]:
#!/usr/bin/env python

import os
import argparse
import functions
import itertools

from pathlib import Path
import numpy as np

In [2]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [3]:
def number_to_str(n):
    if n == 0:
        return 0
    return "{0:.3e}".format(n)
#     if n > 1000:
#         return "{0:.4e}".format(n)
#     else:
#         return "{0:.2f}".format(n)

def print_table(results):
    print("Function &Best &Worst &Median &$c$ &$\\bar\{v\}$ &Mean &Std & FRate\\\\")
    print("\\midrule")
    for k, v in results.items():
        print("{0} &{1}({2}) &{3}({4}) &{5}({6}) &{7} &{8} &{9} &{10} &{11}\\\\".format(k, 
                                                                                         number_to_str(v['best_perf']), 
                                                                                         v['best_consts'], 
                                                                                         number_to_str(v['worst_perf']), 
                                                                                         v['worst_consts'], 
                                                                                         number_to_str(v['median_perf']),
                                                                                         v['median_consts'],
                                                                                         tuple(v['c']),
                                                                                         number_to_str(v['v']),
                                                                                         number_to_str(v['mean']),
                                                                                         number_to_str(v['std']),
                                                                                         v['f_rate']
                                                                                         ))

def print_table_rev(results):
    key_order = [("best_perf", "Best"), ("worst_perf", "Worst"), ("median_perf", "Median"), ("c", "$c$"), ("v", "$\\bar{v}$"), ("mean", "Mean"), ("std", "Std"), ("f_rate", "F Rate")]
    print("\\begin{tabular}{c|cccc}")
    print("\\toprule")
    print("&" + "&".join(results.keys()) + "\\\\")
    print("\\midrule")
    for k, n in key_order:
        if k == "best_perf":
            to_print = [f"{h[0]}({h[1]})" for h in list(zip([str(d["best_perf"]) for d in results.values()], [str(d["best_consts"]) for d in results.values()]))]
            print(n + "&" + "&".join(to_print) + "\\\\")
        elif k == "worst_perf":
            to_print = [f"{h[0]}({h[1]})" for h in list(zip([str(d["worst_perf"]) for d in results.values()], [str(d["worst_consts"]) for d in results.values()]))]
            print(n + "&" + "&".join(to_print) + "\\\\")
        else:
            print(n + "&" + "&".join([str(d[k]) for d in results.values()]) + "\\\\")
    print("\\bottomrule")
    print("\\end{tabular}")

In [4]:
def aggregate_results(_d, _r, _p):
    dimensions = _d
    runs = _r
    logdir_path = _p
    logdir = Path(logdir_path)
    results = dict()
    for c in sorted(listdir_nohidden(logdir)):
        best_list = list()
        idx_best_list = list()
        performances = np.array([])

        function_class = getattr(functions, c)
        function_obj = function_class(dimensions=dimensions)

        # feasibility rate
        fes = 0
        for r in listdir_nohidden(logdir / c):
            if r == "plots":
                continue
            s = np.load(logdir / c/ r / "solutions.npy")
            p = np.load(logdir / c / r / "performances.npy")
            if len(performances) == 0:
                performances = np.array([p])
                solutions = np.array([s])
            else:
                performances = np.concatenate([performances, [p]], axis=0)
                solutions = np.concatenate([solutions, [s]], axis=0)

            # take best of this run
            idx_best = np.unravel_index(np.nanargmin(p), p.shape)
            idx_best_list.append(idx_best)
            best_list.append(p[idx_best])

            feasible_solution = False
            # check if there is at least one feasible solution in this run to compute the feasible rate
            # get all positions where there might be feasible solutions (0 for g and 0/1 for h constraints)
            consts = list(function_class.constraints(None).keys())
            combinations = list(map(list, itertools.product([0, 1], repeat=len(consts))))
            for i, comb in enumerate(combinations):
                for const, (k, item) in zip(consts, enumerate(comb)):
                    if const.startswith('g'):
                        combinations[i][k] = 0
            # remove duplicates
            combinations.sort()
            feasible_indexes = list(l for l,_ in itertools.groupby(combinations))
            for f_idx in feasible_indexes:
                if p[tuple(f_idx)] != np.inf:
                    feasible_solution = True
            if feasible_solution:
                fes += 1

        performances[performances == np.inf] = np.nan

        # get the indexes over the entire data structure, because we need to know which constraints were violated
        best = np.min(best_list)
        idx_best =  np.unravel_index(np.where(performances.flatten() == best)[0][0], performances.shape)
        worst = np.max(best_list)
        idx_worst = np.unravel_index(np.where(performances.flatten() == worst)[0][0], performances.shape)
        idx_median = np.unravel_index(
            np.where(performances.flatten() == np.nanpercentile(performances,50,interpolation='nearest'))[0][0], 
            performances.shape
        )
        median = performances[idx_median]

        mean = np.mean(best_list)
        std = np.std(best_list)

        # compute # of violated constraints
        consts_best = 0
        consts_worst = 0
        consts_median = 0
        consts_median_specific = [0, 0, 0]
        mean_violations = list()
        for k, cost in enumerate(list(function_class.constraints(None).keys())):
            if cost.startswith('g'):
                if idx_best[k+1] > 0:
                    consts_best += 1
                if idx_worst[k+1] > 0:
                    consts_worst += 1
                if idx_median[k+1] > 0:
                    consts_median += 1
                    # add constraint violation to mean_violations
                    mean_violations.append(function_obj.constraints()[cost]['func'](solutions[idx_median]))

            if cost.startswith('h'):
                if idx_best[k+1] > 1:
                    consts_best += 1
                if idx_worst[k+1] > 1:
                    consts_worst += 1
                if idx_median[k+1] > 1:
                    consts_median += 1
                    # add constraint violation to mean_violations
                    mean_violations.append(function_obj.constraints()[cost]['func'](solutions[idx_median]))

            # violation > 0.0001
            if idx_median[k+1] == 2:
                consts_median_specific[0] += 1
            # violation > 0.01
            if idx_median[k+1] == 3:
                consts_median_specific[1] += 1
            # violation > 1.0
            if idx_median[k+1] == 4:
                consts_median_specific[2] += 1

    #     assert sum(consts_median_specific) == consts_median
        results[c] = {
            "best_perf": round(best, 2),
            "best_consts": consts_best,
            "worst_perf": round(worst, 2),
            "worst_consts": consts_worst,
            "median_perf": round(median, 2),
            "median_consts": consts_median,
            "mean": round(mean, 2),
            "std": round(std, 2),
            "c": consts_median_specific,
            "v": round(np.mean(mean_violations), 2) if len(mean_violations) > 0 else 0,
            "f_rate": round(fes / runs, 2)
        }
    print_table(results)

In [5]:
def aggregate_results_cec(_d, _r, _p):
    dimensions = _d
    runs = _r
    logdir_path = _p
    logdir = Path(logdir_path)
    results = dict()
    for c in sorted(listdir_nohidden(logdir)):
        performances = np.array([])

        function_class = getattr(functions, c)
        function_obj = function_class(dimensions=dimensions)

        # feasibility rate
        fes = 0
        best_for_each_run = list()
        best_for_each_run_idxs = list()
        for r in listdir_nohidden(logdir / c):
            if r == "plots":
                continue
            s = np.load(logdir / c/ r / "solutions.npy")
            p = np.load(logdir / c / r / "performances.npy")
            if len(performances) == 0:
                performances = np.array([p])
                solutions = np.array([s])
            else:
                performances = np.concatenate([performances, [p]], axis=0)
                solutions = np.concatenate([solutions, [s]], axis=0)


            feasible_solution = False
            # - Go over all feasible solutions of this run. In case there are feasible solutions, take the best one.
            # - If there are feasible solutions also increase the feasiblity rate.

            # Get all positions where there might be feasible solutions (0 for g and 0/1 for h constraints)
            # We do this by creating all combinations of binary arrays, which indicate the positions of the feasible solutions
            # This is because h constranints are considered feasible also if they are violated in the first bin, so
            # the positions of g constraints must be 0 to be satisfied and the positions of h constraints can be either 0 or 1
            consts = list(function_class.constraints(None).keys())
            combinations = list(map(list, itertools.product([0, 1], repeat=len(consts))))
            for i, comb in enumerate(combinations):
                for const, (k, item) in zip(consts, enumerate(comb)):
                    if const.startswith('g'):
                        combinations[i][k] = 0
            # remove duplicates
            combinations.sort()
            # best feasible solution for this run
            best_sol = np.inf
            best_sol_idx = None
            feasible_indexes = list(l for l,_ in itertools.groupby(combinations))
            for f_idx in feasible_indexes:
                if p[tuple(f_idx)] != np.inf and p[tuple(f_idx)] < best_sol:
                    best_sol = p[tuple(f_idx)]
                    best_sol_idx = tuple(f_idx)
                    feasible_solution = True
            # there was at least one feasible solution
            if feasible_solution:
                fes += 1
            # search for the best UNfeasible solution
            else:            
                # if there is not feasible solution, then we can take the best unfeasible one. 
                # The unfeasible solutions are ordered by the mean value of the violations of all constraints
                # Basically the best unfeasible solution is the one with best v
                # best_sol_idx = np.unravel_index(np.nanargmin(p), p.shape)
                # best_sol = p[best_sol_idx]
                # compute `v` for every element in the map of elites (they are all unfeasible solutions because there are no feasible ones)
                best_unfeasible = np.inf
                best_unfeasible_idx = None
                best_unfeasible_v = np.inf
                for idx, value in np.ndenumerate(p):
                    if value == np.inf:
                        continue
                    current_v = list()
                    for k, cost in enumerate(list(function_class.constraints(None).keys())):
                        if cost.startswith('g') and idx[k] > 0:
                            current_v.append(function_obj.constraints()[cost]['func'](s[idx]))
                        if cost.startswith('h') and idx[k] > 1:
                            current_v.append(function_obj.constraints()[cost]['func'](s[idx]))
                    if len(current_v) == 0:
                        raise ValueError("MeanViolations were not found, but there are no feasible solutions!")
                    current_v = np.mean(current_v)
                    if current_v < best_unfeasible_v:
                        best_unfeasible = value
                        best_unfeasible_idx = idx
                        best_unfeasible_v = current_v
                best_sol = best_unfeasible
                best_sol_idx = best_unfeasible_idx

            best_for_each_run.append(best_sol)
            best_for_each_run_idxs.append(best_sol_idx)


        # ------------------------------------------------------------------------------
        # So now should have the list of best solutions and the correct feasibility rate
        # ------------------------------------------------------------------------------

        performances[performances == np.inf] = np.nan

        # get the indexes over the entire data structure, because we need to know which constraints were violated
        best = np.min(best_for_each_run)
        idx_best =  np.unravel_index(np.where(performances.flatten() == best)[0][0], performances.shape)
        worst = np.max(best_for_each_run)
        idx_worst = np.unravel_index(np.where(performances.flatten() == worst)[0][0], performances.shape)

        idx_median = np.where(np.array(best_for_each_run) == np.nanpercentile(best_for_each_run,50,interpolation='nearest'))[0][0]
        median = best_for_each_run[idx_median]
        # now get the index in the performances structure
        idx_median = np.unravel_index(np.where(performances.flatten() == median)[0][0], performances.shape)

        mean = np.mean(best_for_each_run)
        std = np.std(best_for_each_run)

        # compute # of violated constraints
        consts_best = 0
        consts_worst = 0
        consts_median = 0
        consts_median_specific = [0, 0, 0]
        mean_violations = list()
        for k, cost in enumerate(list(function_class.constraints(None).keys())):
            if cost.startswith('g'):
                if idx_best[k+1] > 0:
                    consts_best += 1
                if idx_worst[k+1] > 0:
                    consts_worst += 1
                if idx_median[k+1] > 0:
                    consts_median += 1
                    # add constraint violation to mean_violations
                    mean_violations.append(function_obj.constraints()[cost]['func'](solutions[idx_median]))

            if cost.startswith('h'):
                if idx_best[k+1] > 1:
                    consts_best += 1
                if idx_worst[k+1] > 1:
                    consts_worst += 1
                if idx_median[k+1] > 1:
                    consts_median += 1
                    # add constraint violation to mean_violations
                    mean_violations.append(function_obj.constraints()[cost]['func'](solutions[idx_median]))

            # violation > 0.0001
            if idx_median[k+1] == 2:
                consts_median_specific[0] += 1
            # violation > 0.01
            if idx_median[k+1] == 3:
                consts_median_specific[1] += 1
            # violation > 1.0
            if idx_median[k+1] == 4:
                consts_median_specific[2] += 1

    #     assert sum(consts_median_specific) == consts_median
        results[c] = {
            "best_perf": round(best, 2),
            "best_consts": consts_best,
            "worst_perf": round(worst, 2),
            "worst_consts": consts_worst,
            "median_perf": round(median, 2),
            "median_consts": consts_median,
            "mean": round(mean, 2),
            "std": round(std, 2),
            "c": consts_median_specific,
            "v": round(np.mean(mean_violations), 2) if len(mean_violations) > 0 else 0,
            "f_rate": round(fes / runs, 2)
        }
    print_table(results)

In [6]:
dimensions = 10
runs = 25
logdir_path = 'log/complete_logs/run_10D_crossover/'

In [7]:
aggregate_results(dimensions, runs, logdir_path)

Function &Best &Worst &Median &$c$ &$\bar\{v\}$ &Mean &Std & FRate\\
\midrule
C01 &-1.310e+00(1) &-4.600e-01(1) &-1.600e-01(1) &(0, 0, 1) &8.000e-01 &-7.300e-01 &2.000e-01 &1.0\\
C02 &-3.390e+00(2) &-2.580e+00(2) &-9.600e-01(2) &(0, 1, 1) &-6.880e+00 &-3.050e+00 &2.200e-01 &0.44\\
C03 &1.154e+14(1) &9.250e+14(1) &4.362e+14(1) &(0, 0, 1) &1.310e+06 &5.005e+14 &2.213e+14 &0.0\\
C04 &-4.441e+01(4) &-6.460e+00(4) &9.600e-01(4) &(0, 1, 3) &5.664e+02 &-3.514e+01 &1.417e+01 &0.0\\
C05 &-5.348e+02(2) &-7.938e+01(2) &-1.681e+02(2) &(0, 0, 2) &-1.972e+01 &-3.187e+02 &1.544e+02 &0.0\\
C06 &-4.336e+02(2) &-1.328e+01(2) &-9.162e+01(2) &(0, 1, 1) &8.514e+01 &-2.528e+02 &1.276e+02 &0.0\\
C07 &6.152e+03(0) &5.908e+07(0) &4.109e+09(1) &(0, 1, 0) &-7.000e-01 &4.070e+06 &1.251e+07 &1.0\\
C08 &2.242e+03(1) &1.388e+06(1) &4.268e+05(0) &(0, 0, 0) &0 &3.925e+05 &4.921e+05 &1.0\\
C09 &9.236e+04(1) &2.809e+13(1) &1.784e+07(0) &(0, 0, 0) &0 &2.474e+12 &7.129e+12 &0.2\\
C10 &2.309e+03(1) &3.539e+13(1) &4.490e+06

In [8]:
aggregate_results_cec(dimensions, runs, logdir_path)

Function &Best &Worst &Median &$c$ &$\bar\{v\}$ &Mean &Std & FRate\\
\midrule
C01 &-8.300e-01(0) &-4.500e-01(0) &-5.500e-01(0) &(0, 0, 0) &0 &-5.700e-01 &1.000e-01 &1.0\\
C02 &-2.140e+00(1) &4.920e+00(0) &2.870e+00(2) &(1, 1, 0) &-8.750e+00 &1.850e+00 &2.230e+00 &0.44\\
C03 &1.154e+14(1) &9.250e+14(1) &4.362e+14(1) &(0, 0, 1) &1.310e+06 &5.005e+14 &2.213e+14 &0.0\\
C04 &-4.441e+01(4) &5.003e+01(4) &3.800e-01(4) &(0, 2, 2) &9.570e+01 &1.270e+00 &2.920e+01 &0.0\\
C05 &-4.801e+02(2) &1.527e+02(1) &-2.316e+02(2) &(0, 1, 1) &1.188e+01 &-2.181e+02 &1.804e+02 &0.0\\
C06 &-4.275e+02(2) &4.210e+02(2) &-1.927e+02(1) &(0, 0, 1) &-1.397e+02 &-1.363e+02 &2.086e+02 &0.0\\
C07 &6.152e+03(0) &5.908e+07(0) &3.171e+05(0) &(0, 0, 0) &0 &4.070e+06 &1.251e+07 &1.0\\
C08 &4.550e+03(0) &1.391e+06(0) &1.518e+05(0) &(0, 0, 0) &0 &4.183e+05 &4.949e+05 &1.0\\
C09 &8.301e+05(1) &1.600e+15(1) &8.227e+07(1) &(0, 1, 0) &-6.704e+01 &1.612e+14 &4.344e+14 &0.2\\
C10 &2.482e+04(1) &7.494e+14(1) &7.169e+07(1) &(1, 0, 0) 